In [1]:
from datetime import datetime
from pathlib import Path

FECHA_HOY = datetime.now().strftime("%Y-%m-%d")
LOG_DIR = Path("D:/trading/logs/staging")
LOG_DIR.mkdir(parents=True, exist_ok=True)
LOG_PATH = LOG_DIR / f"nb_features_{FECHA_HOY}.log"

def log_event(status, mensaje):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(f"{timestamp},{status},{mensaje}\n")
    print(f"[{timestamp}] {status}: {mensaje}")

In [2]:
import pandas as pd
import json

# Leer grupos desde JSON
with open("D:/trading/config/json/symbol_groups.json", "r") as f:
    symbol_groups = json.load(f)

# Unificar simbolos en una lista
simbolos = sorted(set(sum(symbol_groups.values(), [])))
log_event("OK", f"{len(simbolos)} simbolos cargados desde symbol_groups.json")


[2025-05-28 12:13:23] OK: 46 simbolos cargados desde symbol_groups.json


In [3]:
from pathlib import Path
import pandas as pd

INPUT_DIR = Path("D:/trading/data/historic")
OUTPUT_DIR = Path("D:/trading/data/features")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

errores = []

for simbolo in simbolos:
    try:
        archivo = INPUT_DIR / f"{simbolo}.parquet"
        df = pd.read_parquet(archivo)
        df = df.reset_index() if df.index.name == 'datetime' else df

        # === GRUPO 1 === Tendencia
        df['ema_5'] = df['close'].ewm(span=5).mean()
        df['ema_20'] = df['close'].ewm(span=20).mean()
        df['ma_5'] = df['close'].rolling(5).mean()
        df['ma_20'] = df['close'].rolling(20).mean()
        df['ma_ratio_5_20'] = df['ma_5'] / df['ma_20']
        df['slope_ma_5'] = df['ma_5'].diff()
        df['slope_ma_5_std'] = df['slope_ma_5'] / df['ma_5'].rolling(5).std()

        # === GRUPO 2 === Momentum
        df['retorno'] = df['close'].pct_change()
        df['retorno_5d'] = df['close'].pct_change(5)
        df['momentum_10d'] = df['close'] - df['close'].shift(10)
        df['roc_10d'] = df['close'].pct_change(10)

        # === GRUPO 3 === Volatilidad y rango
        df['volatilidad_5d'] = df['retorno'].rolling(5).std()
        df['volatilidad_20d'] = df['retorno'].rolling(20).std()
        df['rango'] = df['high'] - df['low']

        # === GRUPO 5 === Temporales
        if df.index.name == 'datetime':
            df = df.reset_index()
        
        if 'datetime' in df.columns:
            df['fecha'] = pd.to_datetime(df['datetime'])
        elif 'fecha' in df.columns:
            df['fecha'] = pd.to_datetime(df['fecha'])
        else:
            raise KeyError("No se encontró columna de fechas valida ('datetime' o 'fecha')")
        
        df['dia_semana'] = df['fecha'].dt.dayofweek
        df['es_fin_de_mes'] = df['fecha'].dt.is_month_end.astype(int)

        # Guardar salida
        df.to_parquet(OUTPUT_DIR / f"{simbolo}_features.parquet", index=False)
        log_event("OK", f"Features generados para {simbolo}")

    except Exception as e:
        errores.append(simbolo)
        log_event("ERROR", f"{simbolo}: {str(e)}")


[2025-05-28 12:13:23] OK: Features generados para AAPL
[2025-05-28 12:13:23] OK: Features generados para AEP
[2025-05-28 12:13:23] OK: Features generados para AGNCO
[2025-05-28 12:13:23] OK: Features generados para AGNCP
[2025-05-28 12:13:23] OK: Features generados para AMD
[2025-05-28 12:13:23] OK: Features generados para AMZN
[2025-05-28 12:13:23] OK: Features generados para AVGO
[2025-05-28 12:13:23] OK: Features generados para CCI
[2025-05-28 12:13:23] OK: Features generados para CRBG
[2025-05-28 12:13:23] OK: Features generados para CRM
[2025-05-28 12:13:23] OK: Features generados para CTVA
[2025-05-28 12:13:23] OK: Features generados para DGX
[2025-05-28 12:13:23] OK: Features generados para DUK
[2025-05-28 12:13:23] OK: Features generados para ECCF
[2025-05-28 12:13:23] OK: Features generados para ED
[2025-05-28 12:13:23] OK: Features generados para EPD
[2025-05-28 12:13:23] OK: Features generados para EQR
[2025-05-28 12:13:23] OK: Features generados para EW
[2025-05-28 12:13:23

In [4]:
if errores:
    log_event("WARNING", f"{len(errores)} simbolos con errores: {errores}")
else:
    log_event("OK", "Todos los simbolos procesados exitosamente")

[2025-05-28 12:13:24] OK: Todos los simbolos procesados exitosamente
